In [4]:
import pandas as pd
import sqlite3
import os 
import matplotlib.pyplot as plt
import plotly.express as px

## Read in Datasets


In [7]:

advanced_stats = pd.read_csv('advanced_stats.csv')
allgames_stats = pd.read_csv('allgames_stats.csv')
allstar_games_stats = pd.read_csv('allstar_games_stats.csv')
game_highs_stats = pd.read_csv('game_highs_stats.csv')
per_game_stats = pd.read_csv('per_game_stats.csv')
salaries = pd.read_csv('salaries.csv')
totals_stats = pd.read_csv('totals_stats.csv')

datasets = {
    "advanced_stats": advanced_stats,
    "allgames_stats": allgames_stats,
    "allstar_games_stats": allstar_games_stats,
    "game_highs_stats": game_highs_stats,
    "per_game_stats": per_game_stats,
    "salaries": salaries,
    "totals_stats": totals_stats
}


In [9]:

def check_null_values(datasets):
    """
    Check for null values in each dataframe.
    
    Parameters:
    - datasets: dictionary containing dataframe names as keys and dataframes as values.
    
    Returns:
    - A dictionary with dataframe names as keys and a summary of null values as values.
    """
    null_values_dict = {}
    for name, df in datasets.items():
        null_values = df.isnull().sum()
        # Filter out columns with zero null values
        null_values_dict[name] = null_values[null_values > 0].to_dict()
    return null_values_dict

def check_duplicates(datasets):
    """
    Check for duplicate rows in each dataframe.
    
    Parameters:
    - datasets: dictionary containing dataframe names as keys and dataframes as values.
    
    Returns:
    - A dictionary with dataframe names as keys and number of duplicate rows as values.
    """
    duplicates_dict = {}
    for name, df in datasets.items():
        duplicates_dict[name] = df.duplicated().sum()
    return duplicates_dict

# Execute the tests
null_test_results = check_null_values(datasets)
duplicates_test_results = check_duplicates(datasets)

null_test_results, duplicates_test_results

# Convert null values results to a DataFrame
null_values_df = pd.DataFrame.from_dict(null_test_results, orient='index').fillna(0).astype(int)

# Convert duplicates test results to a DataFrame
duplicates_df = pd.DataFrame.from_dict(duplicates_test_results, orient='index', columns=["Duplicate Rows"])

null_values_df, duplicates_df


(                        X  FG  FGA  FG%  3P%  FT  FTA  FT%  ORB  DRB  ...  \
 allgames_stats       2137   4    7   17  504   1    2  135    1    2  ...   
 allstar_games_stats     0   4    4    5   10   4    4   17    4    0  ...   
 game_highs_stats        0   0    0    0    0   0    0    0    0    0  ...   
 
                      GmSc  G  GS  MP  3P  3PA  TRB  AST  TOV  PF  
 allgames_stats        219  0   0   0   0    0    0    0    0   0  
 allstar_games_stats     0  4   3   4   4    4    4    4    4   4  
 game_highs_stats        0  0   0  47   0    0    0    0    0   0  
 
 [3 rows x 23 columns],
                      Duplicate Rows
 advanced_stats                    0
 allgames_stats                    0
 allstar_games_stats               0
 game_highs_stats                  0
 per_game_stats                    0
 salaries                          0
 totals_stats                      0)

In [11]:
# Identifying which datasets have both 'Season' and 'Player' columns
data_with_keys = [df for df in [advanced_stats, allgames_stats, allstar_games_stats, game_highs_stats, per_game_stats, salaries, totals_stats] if "Season" in df.columns and "Player" in df.columns]

dataset_names_with_keys = {
    "advanced_stats": advanced_stats,
    "allgames_stats": allgames_stats,
    "allstar_games_stats": allstar_games_stats,
    "game_highs_stats": game_highs_stats,
    "per_game_stats": per_game_stats,
    "salaries": salaries,
    "totals_stats": totals_stats
}

datasets_with_keys = {name: df for name, df in dataset_names_with_keys.items() if "Season" in df.columns and "Player" in df.columns}

datasets_with_keys.keys()

dict_keys(['advanced_stats', 'allstar_games_stats', 'game_highs_stats', 'per_game_stats', 'salaries', 'totals_stats'])

In [14]:
final_merged_dataset = totals_stats

for name, df in datasets_with_keys.items():
    if name != "totals_stats":
        final_merged_dataset = pd.merge(final_merged_dataset, df, on=["Season", "Player"], how="left", suffixes=('', '_y'))

# Dropping any duplicate columns with '_y' suffix
final_merged_dataset = final_merged_dataset.loc[:,~final_merged_dataset.columns.str.endswith('_y')]

# Displaying the head of the merged dataset
final_merged_dataset.head()

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,GmSc,Team,Salary
0,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,2.6,5.1,0.078,2.2,-0.2,1.9,3.1,39.0,Cleveland Cavaliers,$4018920
1,2004-05,20,CLE,NBA,SF,80,80,3388,795,1684,...,4.6,14.3,0.203,6.9,1.5,8.3,8.8,42.9,Cleveland Cavaliers,$4320360
2,2005-06,21,CLE,NBA,SF,79,79,3361,875,1823,...,4.3,16.3,0.232,7.9,1.4,9.3,9.5,43.8,Cleveland Cavaliers,$4621800
3,2005-06,21,CLE,NBA,SF,79,79,3361,875,1823,...,4.3,16.3,0.232,7.9,1.4,9.3,9.5,43.8,Cleveland Cavaliers,$4621800
4,2005-06,21,CLE,NBA,SF,79,79,3361,875,1823,...,4.3,16.3,0.232,7.9,1.4,9.3,9.5,38.4,Cleveland Cavaliers,$4621800


In [16]:
# Merging the final_merged_dataset with allgames_stats using only 'Player' as a key
merged = pd.merge(final_merged_dataset, allgames_stats, on="Player", how="left", suffixes=('', '_y'))

# Dropping any duplicate columns with '_y' suffix
merged = merged.loc[:,~merged.columns.str.endswith('_y')]

# Displaying the head of the newly merged dataset
merged.head()

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,DBPM,BPM,VORP,GmSc,Team,Salary,Date,X,Opp,Result
0,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,-0.2,1.9,3.1,39.0,Cleveland Cavaliers,$4018920,2003-10-29,@,SAC,L
1,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,-0.2,1.9,3.1,39.0,Cleveland Cavaliers,$4018920,2003-10-30,@,PHO,L
2,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,-0.2,1.9,3.1,39.0,Cleveland Cavaliers,$4018920,2003-11-01,@,POR,L
3,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,-0.2,1.9,3.1,39.0,Cleveland Cavaliers,$4018920,2003-11-05,NaN,DEN,L
4,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,-0.2,1.9,3.1,39.0,Cleveland Cavaliers,$4018920,2003-11-07,@,IND,L


In [17]:
# Merging Test

# Based on the notebook, the merging is done in two stages:
# 1. Merging datasets with both "Season" and "Player" columns
# 2. Further merging the result with allgames_stats using only the 'Player' column

# Stage 1: Merging datasets with both "Season" and "Player" columns
datasets_with_keys = {
    "advanced_stats": advanced_stats,
    "allstar_games_stats": allstar_games_stats,
    "game_highs_stats": game_highs_stats,
    "per_game_stats": per_game_stats,
    "salaries": salaries,
    "totals_stats": totals_stats
}

final_merged_dataset = totals_stats
for name, df in datasets_with_keys.items():
    if name != "totals_stats":
        final_merged_dataset = pd.merge(final_merged_dataset, df, on=["Season", "Player"], how="left", suffixes=('', '_y'))

# Dropping any duplicate columns with '_y' suffix
final_merged_dataset = final_merged_dataset.loc[:,~final_merged_dataset.columns.str.endswith('_y')]

# Stage 2: Merging with allgames_stats using only 'Player' as a key
merged = pd.merge(final_merged_dataset, allgames_stats, on="Player", how="left", suffixes=('', '_y'))
merged = merged.loc[:,~merged.columns.str.endswith('_y')]

# Check the shapes of the resultant datasets
merged_shapes = {
    "final_merged_dataset_shape": final_merged_dataset.shape,
    "merged_shape": merged.shape
}

merged_shapes


{'final_merged_dataset_shape': (666, 55), 'merged_shape': (949587, 59)}

In [18]:
# Creating a SQLite database in memory for the merged dataset
conn_merged = sqlite3.connect(':memory:')

# Writing the merged dataframe to a new table named 'bball_data_merged'
merged.to_sql('bball_data_merged', conn_merged, index=False)

# Verifying that the data has been loaded into the table
pd.read_sql_query("SELECT * FROM bball_data_merged LIMIT 5", conn_merged)

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,DBPM,BPM,VORP,GmSc,Team,Salary,Date,X,Opp,Result
0,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,-0.2,1.9,3.1,39.0,Cleveland Cavaliers,$4018920,2003-10-29,@,SAC,L
1,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,-0.2,1.9,3.1,39.0,Cleveland Cavaliers,$4018920,2003-10-30,@,PHO,L
2,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,-0.2,1.9,3.1,39.0,Cleveland Cavaliers,$4018920,2003-11-01,@,POR,L
3,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,-0.2,1.9,3.1,39.0,Cleveland Cavaliers,$4018920,2003-11-05,None,DEN,L
4,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,-0.2,1.9,3.1,39.0,Cleveland Cavaliers,$4018920,2003-11-07,@,IND,L


### Queries

In [21]:
# Query 1: Average number of games played by players
query_1 = """
SELECT AVG(G) AS Average_Games_Played
FROM bball_data_merged;
"""

avg_games_played = pd.read_sql_query(query_1, conn_merged)
avg_games_played


,Average_Games_Played
0,44.767911


In [25]:
# Query: Players and the year they made the most money in their careers
max_salary_query = """
SELECT Player, Season AS Year, MAX(Salary) AS Highest_Salary
FROM bball_data_merged
WHERE Player IN ('Michael Jordan', 'Kobe Bryant')
GROUP BY Player
ORDER BY Highest_Salary DESC
LIMIT 20;
"""

players_max_salary = pd.read_sql_query(max_salary_query, conn_merged)
players_max_salary

,Player,Year,Highest_Salary
0,Kobe Bryant,1999-00,$9000000
1,Michael Jordan,1987-88,$845000


In [28]:
# Career Points
query_career_points = """
SELECT Player, SUM(PTS) AS Total_Points
FROM bball_data_merged
WHERE Player IN ('Michael Jordan', 'Kobe Bryant')
GROUP BY Player;
"""

career_points = pd.read_sql_query(query_career_points, conn_merged)
career_points

,Player,Total_Points
0,Kobe Bryant,426606588
1,Michael Jordan,356693877


In [30]:
# Average Points per Game
query_avg_points_per_game = """
SELECT Player, AVG(`PTS`) AS Average_Points_Per_Game
FROM bball_data_merged
WHERE Player IN ('Michael Jordan', 'Kobe Bryant', 'LeBron James')
GROUP BY Player;
"""

avg_points_per_game = pd.read_sql_query(query_avg_points_per_game, conn_merged)
avg_points_per_game

,Player,Average_Points_Per_Game
0,Kobe Bryant,1111.910204
1,Michael Jordan,1357.747619


In [ ]:
# Corrected Query 2: Average Points per Game using PTS and G columns
query_avg_points_per_game_calculated = """
SELECT Player, SUM(PTS) / SUM(G) AS Average_Points_Per_Game
FROM bball_data_merged
WHERE Player IN ('Michael Jordan', 'Kobe Bryant', 'LeBron James')
GROUP BY Player;
"""

avg_points_per_game_calculated = pd.read_sql_query(query_avg_points_per_game_calculated, conn_merged)
avg_points_per_game_calculated

In [17]:
# Query 3: Best Seasons by Points
query_best_season_points = """
SELECT Player, Season AS Year, MAX(PTS) AS Most_Points
FROM bball_data_merged
WHERE Player IN ('Michael Jordan', 'Kobe Bryant', 'LeBron James')
GROUP BY Player;
"""

best_season_points = pd.read_sql_query(query_best_season_points, conn_merged)
best_season_points

,Player,Year,Most_Points
0,Kobe Bryant,2005-06,2832
1,Michael Jordan,1986-87,3041


In [23]:
# Query 4: Career Assists and Rebounds
query_career_assists_rebounds = """
SELECT Player, SUM(AST) AS Total_Assists, SUM(TRB) AS Total_Rebounds
FROM bball_data_merged
WHERE Player IN ('Michael Jordan', 'Kobe Bryant', 'LeBron James')
GROUP BY Player;
"""

career_assists_rebounds = pd.read_sql_query(query_career_assists_rebounds, conn_merged)
career_assists_rebounds

,Player,Total_Assists,Total_Rebounds
0,Kobe Bryant,77933556,88085934
1,Michael Jordan,61165143,70981740


In [25]:
# Query 5: Most Valuable Seasons using WS (Win Shares) metric
query_most_valuable_season = """
SELECT Player, Season AS Year, MAX(WS) AS Highest_Win_Shares
FROM bball_data_merged
WHERE Player IN ('Michael Jordan', 'Kobe Bryant', 'LeBron James')
GROUP BY Player;
"""

most_valuable_season = pd.read_sql_query(query_most_valuable_season, conn_merged)
most_valuable_season


,Player,Year,Highest_Win_Shares
0,Kobe Bryant,2005-06,15.3
1,Michael Jordan,1987-88,21.2


In [26]:
# Query 6: Efficiency Metrics
query_efficiency_metrics = """
SELECT Player, 
       AVG(`FG%`) AS Average_Field_Goal_Percentage, 
       AVG(`3P%`) AS Average_Three_Point_Percentage, 
       AVG(`FT%`) AS Average_Free_Throw_Percentage
FROM bball_data_merged
WHERE Player IN ('Michael Jordan', 'Kobe Bryant', 'LeBron James')
GROUP BY Player;
"""

efficiency_metrics = pd.read_sql_query(query_efficiency_metrics, conn_merged)
efficiency_metrics


,Player,Average_Field_Goal_Percentage,Average_Three_Point_Percentage,Average_Free_Throw_Percentage
0,Kobe Bryant,0.446710,0.325931,0.825237
1,Michael Jordan,0.489243,0.332533,0.835557


In [28]:
# Query 7: Defensive Contributions
query_defensive_contributions = """
SELECT Player, 
       AVG(STL) AS Average_Steals, 
       AVG(BLK) AS Average_Blocks
FROM bball_data_merged
WHERE Player IN ('Michael Jordan', 'Kobe Bryant', 'LeBron James')
GROUP BY Player;
"""

defensive_contributions = pd.read_sql_query(query_defensive_contributions, conn_merged)
defensive_contributions


,Player,Average_Steals,Average_Blocks
0,Kobe Bryant,64.800000,22.771429
1,Michael Jordan,103.161905,37.871429


In [31]:
# Redefining the list of SQL queries

queries = [
    """SELECT AVG(G) AS Average_Games_Played FROM bball_data_merged;""",
    """SELECT Player, Season AS Year, MAX(Salary) AS Highest_Salary FROM bball_data_merged GROUP BY Player ORDER BY Highest_Salary DESC LIMIT 20;""",
    """SELECT Player, Season AS Year, MAX(PTS) AS Most_Points FROM bball_data_merged WHERE Player IN ('Michael Jordan', 'Kobe Bryant', 'LeBron James') GROUP BY Player;""",
    """SELECT Player, SUM(AST) AS Total_Assists, SUM(TRB) AS Total_Rebounds FROM bball_data_merged WHERE Player IN ('Michael Jordan', 'Kobe Bryant', 'LeBron James') GROUP BY Player;""",
    """SELECT Player, Season AS Year, MAX(WS) AS Highest_Win_Shares FROM bball_data_merged WHERE Player IN ('Michael Jordan', 'Kobe Bryant', 'LeBron James') GROUP BY Player;""",
    """SELECT Player, AVG(`FG%`) AS Average_Field_Goal_Percentage, AVG(`3P%`) AS Average_Three_Point_Percentage, AVG(`FT%`) AS Average_Free_Throw_Percentage FROM bball_data_merged WHERE Player IN ('Michael Jordan', 'Kobe Bryant', 'LeBron James') GROUP BY Player;""",
    """SELECT Player, AVG(STL) AS Average_Steals, AVG(BLK) AS Average_Blocks FROM bball_data_merged WHERE Player IN ('Michael Jordan', 'Kobe Bryant', 'LeBron James') GROUP BY Player;"""
]

# Execute the SQL queries and store results
query_results_chunked = {}
for index, query in enumerate(queries, 1):
    try:
        result = pd.read_sql_query(query, conn_merged)
        query_results_chunked[f"Query_{index}"] = "Executed Successfully"
    except Exception as e:
        query_results_chunked[f"Query_{index}"] = f"Error: {str(e)}"

query_results_chunked


{'Query_1': 'Executed Successfully',
 'Query_2': 'Executed Successfully',
 'Query_3': 'Executed Successfully',
 'Query_4': 'Executed Successfully',
 'Query_5': 'Executed Successfully',
 'Query_6': 'Executed Successfully',
 'Query_7': 'Executed Successfully'}